# Canadian trade data

This notebook downloads raw trade data from [ISED Canada](https://ised-isde.canada.ca/site/trade-data-online/en)

Includes trade balance data by province/trade-partner and [HS2](https://en.wikipedia.org/wiki/Harmonized_System) product categorization.



In [ ]:
# TODO
# download SUT tables: https://www150.statcan.gc.ca/n1/en/catalogue/15-602-X

In [2]:
import requests
import os
from datetime import datetime as dt

In [3]:
raw_dir = os.path.abspath('../raw/')

## Analyze links

In [ ]:
# parse url parameters
link = 'https://ised-isde.canada.ca/app/ixb/tdo/runRpt.html?cssIncludes=%2Fcss%2Fcommon.css&cssIncludes=%2Fcss%2Fadd_WET_4-0_Canada_Apps.css&jsIncludes=js%2Futils.js&jsIncludes=js%2FcodeValidation.js&jsIncludes=js%2FdropdownFiltering.js&jsIncludes=js%2FcriteriaFormCodeSearch.js&jsIncludes=js%2FcriteriaFormCountryRegionState.js&jsIncludes=js%2FchangeCriteria.js&jsIncludes=js%2FselectedCodeDescriptions.js&jsIncludes=js%2Fsiteimprove.js&jsIncludes=js%2FgoogleTagManager.js&jsIncludes=js%2FsurveyPopup.js&grouped=INDIVIDUAL&searchType=BL&areaCodes=&naArea=P48&countryList=DET&toFromCountry=CDN&reportType=TB&customYears=2024&periodString=&timePeriod=%7CCustom+Years&currency=CDN&lang=&productType=HS6&hSelectedCodes=%7C270710'

def parse_link(link):
    """Parse the URL parameters into a dictionary."""
    # Split the link into header and parameters
    
    header, params = link.split('?')
    params = params.split('&')
    params = [p.split('=') for p in params]
    params = {k:v for k, v in params}
    return params

link = 'https://ised-isde.canada.ca/app/ixb/tdo/runRpt.html?cssIncludes=%2Fcss%2Fcommon.css&cssIncludes=%2Fcss%2Fadd_WET_4-0_Canada_Apps.css&jsIncludes=js%2Futils.js&jsIncludes=js%2FcodeValidation.js&jsIncludes=js%2FdropdownFiltering.js&jsIncludes=js%2FcriteriaFormCodeSearch.js&jsIncludes=js%2FcriteriaFormCountryRegionState.js&jsIncludes=js%2FchangeCriteria.js&jsIncludes=js%2FselectedCodeDescriptions.js&jsIncludes=js%2Fsiteimprove.js&jsIncludes=js%2FgoogleTagManager.js&jsIncludes=js%2FsurveyPopup.js&grouped=GROUPED&searchType=BL&areaCodes=&naArea=9999&countryList=ALL&toFromCountry=CDN&reportType=TB&customYears=2024&periodString=&timePeriod=%7CCustom+Years&currency=CDN&lang=&productType=HS6&hSelectedCodes=%7CA%7CB%7CC%7CD%7CE%7CF%7CG%7CH%7CI%7CJ%7CK%7CL%7CM%7CN%7CO%7CP%7CQ%7CR%7CS%7CT%7CU'
parse_link(link)

In [7]:
links = '''
https://ised-isde.canada.ca/app/ixb/tdo/runRpt.html?cssIncludes=%2Fcss%2Fcommon.css&cssIncludes=%2Fcss%2Fadd_WET_4-0_Canada_Apps.css&jsIncludes=js%2Futils.js&jsIncludes=js%2FcodeValidation.js&jsIncludes=js%2FdropdownFiltering.js&jsIncludes=js%2FcriteriaFormCodeSearch.js&jsIncludes=js%2FcriteriaFormCountryRegionState.js&jsIncludes=js%2FchangeCriteria.js&jsIncludes=js%2FselectedCodeDescriptions.js&jsIncludes=js%2Fsiteimprove.js&jsIncludes=js%2FgoogleTagManager.js&jsIncludes=js%2FsurveyPopup.js&grouped=GROUPED&searchType=All&areaCodes=&naArea=9999&countryList=DET&toFromCountry=CDN&reportType=TB&customYears=2024&periodString=&timePeriod=%7CCustom+Years&currency=CDN&lang=&productType=HS6&hSelectedCodes=%7CA%7CB%7CC%7CD%7CE%7CF%7CG%7CH%7CI%7CJ%7CK%7CL%7CM%7CN%7CO%7CP%7CQ%7CR%7CS%7CT%7CU
https://ised-isde.canada.ca/app/ixb/tdo/runRpt.html?cssIncludes=%2Fcss%2Fcommon.css&cssIncludes=%2Fcss%2Fadd_WET_4-0_Canada_Apps.css&jsIncludes=js%2Futils.js&jsIncludes=js%2FcodeValidation.js&jsIncludes=js%2FdropdownFiltering.js&jsIncludes=js%2FcriteriaFormCodeSearch.js&jsIncludes=js%2FcriteriaFormCountryRegionState.js&jsIncludes=js%2FchangeCriteria.js&jsIncludes=js%2FselectedCodeDescriptions.js&jsIncludes=js%2Fsiteimprove.js&jsIncludes=js%2FgoogleTagManager.js&jsIncludes=js%2FsurveyPopup.js&grouped=GROUPED&searchType=All&areaCodes=&naArea=P10&countryList=DET&toFromCountry=CDN&reportType=TB&customYears=2024&periodString=&timePeriod=%7CCustom+Years&currency=CDN&lang=&productType=HS6&hSelectedCodes=%7CA%7CB%7CC%7CD%7CE%7CF%7CG%7CH%7CI%7CJ%7CK%7CL%7CM%7CN%7CO%7CP%7CQ%7CR%7CS%7CT%7CU
https://ised-isde.canada.ca/app/ixb/tdo/runRpt.html?cssIncludes=%2Fcss%2Fcommon.css&cssIncludes=%2Fcss%2Fadd_WET_4-0_Canada_Apps.css&jsIncludes=js%2Futils.js&jsIncludes=js%2FcodeValidation.js&jsIncludes=js%2FdropdownFiltering.js&jsIncludes=js%2FcriteriaFormCodeSearch.js&jsIncludes=js%2FcriteriaFormCountryRegionState.js&jsIncludes=js%2FchangeCriteria.js&jsIncludes=js%2FselectedCodeDescriptions.js&jsIncludes=js%2Fsiteimprove.js&jsIncludes=js%2FgoogleTagManager.js&jsIncludes=js%2FsurveyPopup.js&grouped=GROUPED&searchType=All&areaCodes=&naArea=P11&countryList=DET&toFromCountry=CDN&reportType=TB&customYears=2024&periodString=&timePeriod=%7CCustom+Years&currency=CDN&lang=&productType=HS6&hSelectedCodes=%7CA%7CB%7CC%7CD%7CE%7CF%7CG%7CH%7CI%7CJ%7CK%7CL%7CM%7CN%7CO%7CP%7CQ%7CR%7CS%7CT%7CU
https://ised-isde.canada.ca/app/ixb/tdo/runRpt.html?cssIncludes=%2Fcss%2Fcommon.css&cssIncludes=%2Fcss%2Fadd_WET_4-0_Canada_Apps.css&jsIncludes=js%2Futils.js&jsIncludes=js%2FcodeValidation.js&jsIncludes=js%2FdropdownFiltering.js&jsIncludes=js%2FcriteriaFormCodeSearch.js&jsIncludes=js%2FcriteriaFormCountryRegionState.js&jsIncludes=js%2FchangeCriteria.js&jsIncludes=js%2FselectedCodeDescriptions.js&jsIncludes=js%2Fsiteimprove.js&jsIncludes=js%2FgoogleTagManager.js&jsIncludes=js%2FsurveyPopup.js&grouped=GROUPED&searchType=All&areaCodes=&naArea=P12&countryList=DET&toFromCountry=CDN&reportType=TB&customYears=2024&periodString=&timePeriod=%7CCustom+Years&currency=CDN&lang=&productType=HS6&hSelectedCodes=%7CA%7CB%7CC%7CD%7CE%7CF%7CG%7CH%7CI%7CJ%7CK%7CL%7CM%7CN%7CO%7CP%7CQ%7CR%7CS%7CT%7CU
https://ised-isde.canada.ca/app/ixb/tdo/runRpt.html?cssIncludes=%2Fcss%2Fcommon.css&cssIncludes=%2Fcss%2Fadd_WET_4-0_Canada_Apps.css&jsIncludes=js%2Futils.js&jsIncludes=js%2FcodeValidation.js&jsIncludes=js%2FdropdownFiltering.js&jsIncludes=js%2FcriteriaFormCodeSearch.js&jsIncludes=js%2FcriteriaFormCountryRegionState.js&jsIncludes=js%2FchangeCriteria.js&jsIncludes=js%2FselectedCodeDescriptions.js&jsIncludes=js%2Fsiteimprove.js&jsIncludes=js%2FgoogleTagManager.js&jsIncludes=js%2FsurveyPopup.js&grouped=GROUPED&searchType=All&areaCodes=&naArea=P13&countryList=DET&toFromCountry=CDN&reportType=TB&customYears=2024&periodString=&timePeriod=%7CCustom+Years&currency=CDN&lang=&productType=HS6&hSelectedCodes=%7CA%7CB%7CC%7CD%7CE%7CF%7CG%7CH%7CI%7CJ%7CK%7CL%7CM%7CN%7CO%7CP%7CQ%7CR%7CS%7CT%7CU
https://ised-isde.canada.ca/app/ixb/tdo/runRpt.html?cssIncludes=%2Fcss%2Fcommon.css&cssIncludes=%2Fcss%2Fadd_WET_4-0_Canada_Apps.css&jsIncludes=js%2Futils.js&jsIncludes=js%2FcodeValidation.js&jsIncludes=js%2FdropdownFiltering.js&jsIncludes=js%2FcriteriaFormCodeSearch.js&jsIncludes=js%2FcriteriaFormCountryRegionState.js&jsIncludes=js%2FchangeCriteria.js&jsIncludes=js%2FselectedCodeDescriptions.js&jsIncludes=js%2Fsiteimprove.js&jsIncludes=js%2FgoogleTagManager.js&jsIncludes=js%2FsurveyPopup.js&grouped=GROUPED&searchType=All&areaCodes=&naArea=P24&countryList=DET&toFromCountry=CDN&reportType=TB&customYears=2024&periodString=&timePeriod=%7CCustom+Years&currency=CDN&lang=&productType=HS6&hSelectedCodes=%7CA%7CB%7CC%7CD%7CE%7CF%7CG%7CH%7CI%7CJ%7CK%7CL%7CM%7CN%7CO%7CP%7CQ%7CR%7CS%7CT%7CU
https://ised-isde.canada.ca/app/ixb/tdo/runRpt.html?cssIncludes=%2Fcss%2Fcommon.css&cssIncludes=%2Fcss%2Fadd_WET_4-0_Canada_Apps.css&jsIncludes=js%2Futils.js&jsIncludes=js%2FcodeValidation.js&jsIncludes=js%2FdropdownFiltering.js&jsIncludes=js%2FcriteriaFormCodeSearch.js&jsIncludes=js%2FcriteriaFormCountryRegionState.js&jsIncludes=js%2FchangeCriteria.js&jsIncludes=js%2FselectedCodeDescriptions.js&jsIncludes=js%2Fsiteimprove.js&jsIncludes=js%2FgoogleTagManager.js&jsIncludes=js%2FsurveyPopup.js&grouped=GROUPED&searchType=All&areaCodes=&naArea=P35&countryList=DET&toFromCountry=CDN&reportType=TB&customYears=2024&periodString=&timePeriod=%7CCustom+Years&currency=CDN&lang=&productType=HS6&hSelectedCodes=%7CA%7CB%7CC%7CD%7CE%7CF%7CG%7CH%7CI%7CJ%7CK%7CL%7CM%7CN%7CO%7CP%7CQ%7CR%7CS%7CT%7CU
https://ised-isde.canada.ca/app/ixb/tdo/runRpt.html?cssIncludes=%2Fcss%2Fcommon.css&cssIncludes=%2Fcss%2Fadd_WET_4-0_Canada_Apps.css&jsIncludes=js%2Futils.js&jsIncludes=js%2FcodeValidation.js&jsIncludes=js%2FdropdownFiltering.js&jsIncludes=js%2FcriteriaFormCodeSearch.js&jsIncludes=js%2FcriteriaFormCountryRegionState.js&jsIncludes=js%2FchangeCriteria.js&jsIncludes=js%2FselectedCodeDescriptions.js&jsIncludes=js%2Fsiteimprove.js&jsIncludes=js%2FgoogleTagManager.js&jsIncludes=js%2FsurveyPopup.js&grouped=GROUPED&searchType=All&areaCodes=&naArea=P46&countryList=DET&toFromCountry=CDN&reportType=TB&customYears=2024&periodString=&timePeriod=%7CCustom+Years&currency=CDN&lang=&productType=HS6&hSelectedCodes=%7CA%7CB%7CC%7CD%7CE%7CF%7CG%7CH%7CI%7CJ%7CK%7CL%7CM%7CN%7CO%7CP%7CQ%7CR%7CS%7CT%7CU
https://ised-isde.canada.ca/app/ixb/tdo/runRpt.html?cssIncludes=%2Fcss%2Fcommon.css&cssIncludes=%2Fcss%2Fadd_WET_4-0_Canada_Apps.css&jsIncludes=js%2Futils.js&jsIncludes=js%2FcodeValidation.js&jsIncludes=js%2FdropdownFiltering.js&jsIncludes=js%2FcriteriaFormCodeSearch.js&jsIncludes=js%2FcriteriaFormCountryRegionState.js&jsIncludes=js%2FchangeCriteria.js&jsIncludes=js%2FselectedCodeDescriptions.js&jsIncludes=js%2Fsiteimprove.js&jsIncludes=js%2FgoogleTagManager.js&jsIncludes=js%2FsurveyPopup.js&grouped=GROUPED&searchType=All&areaCodes=&naArea=P47&countryList=DET&toFromCountry=CDN&reportType=TB&customYears=2024&periodString=&timePeriod=%7CCustom+Years&currency=CDN&lang=&productType=HS6&hSelectedCodes=%7CA%7CB%7CC%7CD%7CE%7CF%7CG%7CH%7CI%7CJ%7CK%7CL%7CM%7CN%7CO%7CP%7CQ%7CR%7CS%7CT%7CU
https://ised-isde.canada.ca/app/ixb/tdo/runRpt.html?cssIncludes=%2Fcss%2Fcommon.css&cssIncludes=%2Fcss%2Fadd_WET_4-0_Canada_Apps.css&jsIncludes=js%2Futils.js&jsIncludes=js%2FcodeValidation.js&jsIncludes=js%2FdropdownFiltering.js&jsIncludes=js%2FcriteriaFormCodeSearch.js&jsIncludes=js%2FcriteriaFormCountryRegionState.js&jsIncludes=js%2FchangeCriteria.js&jsIncludes=js%2FselectedCodeDescriptions.js&jsIncludes=js%2Fsiteimprove.js&jsIncludes=js%2FgoogleTagManager.js&jsIncludes=js%2FsurveyPopup.js&grouped=GROUPED&searchType=All&areaCodes=&naArea=P48&countryList=DET&toFromCountry=CDN&reportType=TB&customYears=2024&periodString=&timePeriod=%7CCustom+Years&currency=CDN&lang=&productType=HS6&hSelectedCodes=%7CA%7CB%7CC%7CD%7CE%7CF%7CG%7CH%7CI%7CJ%7CK%7CL%7CM%7CN%7CO%7CP%7CQ%7CR%7CS%7CT%7CU
https://ised-isde.canada.ca/app/ixb/tdo/runRpt.html?cssIncludes=%2Fcss%2Fcommon.css&cssIncludes=%2Fcss%2Fadd_WET_4-0_Canada_Apps.css&jsIncludes=js%2Futils.js&jsIncludes=js%2FcodeValidation.js&jsIncludes=js%2FdropdownFiltering.js&jsIncludes=js%2FcriteriaFormCodeSearch.js&jsIncludes=js%2FcriteriaFormCountryRegionState.js&jsIncludes=js%2FchangeCriteria.js&jsIncludes=js%2FselectedCodeDescriptions.js&jsIncludes=js%2Fsiteimprove.js&jsIncludes=js%2FgoogleTagManager.js&jsIncludes=js%2FsurveyPopup.js&grouped=GROUPED&searchType=All&areaCodes=&naArea=P59&countryList=DET&toFromCountry=CDN&reportType=TB&customYears=2024&periodString=&timePeriod=%7CCustom+Years&currency=CDN&lang=&productType=HS6&hSelectedCodes=%7CA%7CB%7CC%7CD%7CE%7CF%7CG%7CH%7CI%7CJ%7CK%7CL%7CM%7CN%7CO%7CP%7CQ%7CR%7CS%7CT%7CU
https://ised-isde.canada.ca/app/ixb/tdo/runRpt.html?cssIncludes=%2Fcss%2Fcommon.css&cssIncludes=%2Fcss%2Fadd_WET_4-0_Canada_Apps.css&jsIncludes=js%2Futils.js&jsIncludes=js%2FcodeValidation.js&jsIncludes=js%2FdropdownFiltering.js&jsIncludes=js%2FcriteriaFormCodeSearch.js&jsIncludes=js%2FcriteriaFormCountryRegionState.js&jsIncludes=js%2FchangeCriteria.js&jsIncludes=js%2FselectedCodeDescriptions.js&jsIncludes=js%2Fsiteimprove.js&jsIncludes=js%2FgoogleTagManager.js&jsIncludes=js%2FsurveyPopup.js&grouped=GROUPED&searchType=All&areaCodes=&naArea=P61&countryList=DET&toFromCountry=CDN&reportType=TB&customYears=2024&periodString=&timePeriod=%7CCustom+Years&currency=CDN&lang=&productType=HS6&hSelectedCodes=%7CA%7CB%7CC%7CD%7CE%7CF%7CG%7CH%7CI%7CJ%7CK%7CL%7CM%7CN%7CO%7CP%7CQ%7CR%7CS%7CT%7CU
https://ised-isde.canada.ca/app/ixb/tdo/runRpt.html?cssIncludes=%2Fcss%2Fcommon.css&cssIncludes=%2Fcss%2Fadd_WET_4-0_Canada_Apps.css&jsIncludes=js%2Futils.js&jsIncludes=js%2FcodeValidation.js&jsIncludes=js%2FdropdownFiltering.js&jsIncludes=js%2FcriteriaFormCodeSearch.js&jsIncludes=js%2FcriteriaFormCountryRegionState.js&jsIncludes=js%2FchangeCriteria.js&jsIncludes=js%2FselectedCodeDescriptions.js&jsIncludes=js%2Fsiteimprove.js&jsIncludes=js%2FgoogleTagManager.js&jsIncludes=js%2FsurveyPopup.js&grouped=GROUPED&searchType=All&areaCodes=&naArea=P60&countryList=DET&toFromCountry=CDN&reportType=TB&customYears=2024&periodString=&timePeriod=%7CCustom+Years&currency=CDN&lang=&productType=HS6&hSelectedCodes=%7CA%7CB%7CC%7CD%7CE%7CF%7CG%7CH%7CI%7CJ%7CK%7CL%7CM%7CN%7CO%7CP%7CQ%7CR%7CS%7CT%7CU
https://ised-isde.canada.ca/app/ixb/tdo/runRpt.html?cssIncludes=%2Fcss%2Fcommon.css&cssIncludes=%2Fcss%2Fadd_WET_4-0_Canada_Apps.css&jsIncludes=js%2Futils.js&jsIncludes=js%2FcodeValidation.js&jsIncludes=js%2FdropdownFiltering.js&jsIncludes=js%2FcriteriaFormCodeSearch.js&jsIncludes=js%2FcriteriaFormCountryRegionState.js&jsIncludes=js%2FchangeCriteria.js&jsIncludes=js%2FselectedCodeDescriptions.js&jsIncludes=js%2Fsiteimprove.js&jsIncludes=js%2FgoogleTagManager.js&jsIncludes=js%2FsurveyPopup.js&grouped=GROUPED&searchType=All&areaCodes=&naArea=P62&countryList=DET&toFromCountry=CDN&reportType=TB&customYears=2024&periodString=&timePeriod=%7CCustom+Years&currency=CDN&lang=&productType=HS6&hSelectedCodes=%7CA%7CB%7CC%7CD%7CE%7CF%7CG%7CH%7CI%7CJ%7CK%7CL%7CM%7CN%7CO%7CP%7CQ%7CR%7CS%7CT%7CU
'''.split()

na_areas = []
for link in links:

    params = parse_link(link)
    print(params)
    na_areas.append(params['naArea'])

print('na_areas = ', na_areas)

{'cssIncludes': '%2Fcss%2Fadd_WET_4-0_Canada_Apps.css', 'jsIncludes': 'js%2FsurveyPopup.js', 'grouped': 'GROUPED', 'searchType': 'All', 'areaCodes': '', 'naArea': '9999', 'countryList': 'DET', 'toFromCountry': 'CDN', 'reportType': 'TB', 'customYears': '2024', 'periodString': '', 'timePeriod': '%7CCustom+Years', 'currency': 'CDN', 'lang': '', 'productType': 'HS6', 'hSelectedCodes': '%7CA%7CB%7CC%7CD%7CE%7CF%7CG%7CH%7CI%7CJ%7CK%7CL%7CM%7CN%7CO%7CP%7CQ%7CR%7CS%7CT%7CU'}
{'cssIncludes': '%2Fcss%2Fadd_WET_4-0_Canada_Apps.css', 'jsIncludes': 'js%2FsurveyPopup.js', 'grouped': 'GROUPED', 'searchType': 'All', 'areaCodes': '', 'naArea': 'P10', 'countryList': 'DET', 'toFromCountry': 'CDN', 'reportType': 'TB', 'customYears': '2024', 'periodString': '', 'timePeriod': '%7CCustom+Years', 'currency': 'CDN', 'lang': '', 'productType': 'HS6', 'hSelectedCodes': '%7CA%7CB%7CC%7CD%7CE%7CF%7CG%7CH%7CI%7CJ%7CK%7CL%7CM%7CN%7CO%7CP%7CQ%7CR%7CS%7CT%7CU'}
{'cssIncludes': '%2Fcss%2Fadd_WET_4-0_Canada_Apps.css', '

In [8]:
link = 'https://ised-isde.canada.ca/app/ixb/tdo/report.csv?cssIncludes=%2Fcss%2Fcommon.css&cssIncludes=%2Fcss%2Fadd_WET_4-0_Canada_Apps.css&jsIncludes=js%2Futils.js&jsIncludes=js%2FcodeValidation.js&jsIncludes=js%2FdropdownFiltering.js&jsIncludes=js%2FcriteriaFormCodeSearch.js&jsIncludes=js%2FcriteriaFormCountryRegionState.js&jsIncludes=js%2FchangeCriteria.js&jsIncludes=js%2FselectedCodeDescriptions.js&jsIncludes=js%2Fsiteimprove.js&jsIncludes=js%2FgoogleTagManager.js&jsIncludes=js%2FsurveyPopup.js&grouped=GROUPED&searchType=All&areaCodes=&naArea=P59&countryList=ALL&toFromCountry=CDN&reportType=TB&customYears=2024&periodString=&timePeriod=%7CCustom+Years&currency=CDN&lang=&productType=HS6&hSelectedCodes=%7CA%7CB%7CC%7CD%7CE%7CF%7CG%7CH%7CI%7CJ%7CK%7CL%7CM%7CN%7CO%7CP%7CQ%7CR%7CS%7CT%7CU'
parse_link(link)

{'cssIncludes': '%2Fcss%2Fadd_WET_4-0_Canada_Apps.css',
 'jsIncludes': 'js%2FsurveyPopup.js',
 'grouped': 'GROUPED',
 'searchType': 'All',
 'areaCodes': '',
 'naArea': 'P59',
 'countryList': 'ALL',
 'toFromCountry': 'CDN',
 'reportType': 'TB',
 'customYears': '2024',
 'periodString': '',
 'timePeriod': '%7CCustom+Years',
 'currency': 'CDN',
 'lang': '',
 'productType': 'HS6',
 'hSelectedCodes': '%7CA%7CB%7CC%7CD%7CE%7CF%7CG%7CH%7CI%7CJ%7CK%7CL%7CM%7CN%7CO%7CP%7CQ%7CR%7CS%7CT%7CU'}

## Download

In [9]:
start, end = 1990, 2024

# years
years = [str(x) for x in range(start, end+1)]

# product codes
codes = ['C'+chr(n) for n in range(ord('A'), ord('U')+1)] # chapter codes
# codes = [str(x).zfill(2) for x in range(1, 100)] # HS2 codes

na_areas =  {
    '9999' : 'CA',
    'P10' : 'NL',
    'P11' : 'PE',
    'P12' : 'NS',
    'P13' : 'NB',
    'P24' : 'QC',
    'P35' : 'ON',
    'P46' : 'MB',
    'P47' : 'SK',
    'P48' : 'AB',
    'P59' : 'BC',
    'P61' : 'NT',
    'P60' : 'YT',
    'P62' : 'NU',
    }

url = 'https://ised-isde.canada.ca/app/ixb/tdo/report.csv'

# request parameters
params = {
    'cssIncludes': '%2Fcss%2Fadd_WET_4-0_Canada_Apps.css',
    'jsIncludes': 'js%2FsurveyPopup.js',
    'grouped': 'GROUPED',
    'searchType': 'All',
    'countryList': 'DET',
    'areaCodes': '',
    'toFromCountry': 'CDN',
    'reportType': 'TB',
    'periodString': '',
    'timePeriod': '%7CCustom+Years',
    'currency': 'CDN',
    'lang': '',
    'productType': 'HS6',
}

In [ ]:
n = 0
N = len(na_areas) * len(years) * len(codes)
t0 = dt.now()

for na_code in na_areas:

    for year in years:

        for code in codes:
            
            n += 1
            print(f'[{n:,} of {N:,}]')
            
            # destination file
            area_abbrev = na_areas[na_code]
            dst = os.path.join(raw_dir, f'{area_abbrev}-{year}-{code}.csv')
            if os.path.exists(dst):
                print('Skipped:', dst)
                continue

            # update parameters
            params['naArea'] = na_code
            params['customYears'] = year
            params['hSelectedCodes'] = code

            resp = requests.get(url, params=params)

            if resp:
                with open(dst, 'wb') as f:
                    f.write(resp.content)
            else:
                print(f'Error: {resp}')
                continue

            # update message
            t1 = dt.now()
            delta = t1 - t0
            avg = delta / n
            eta = avg * (N-n)

            print(f'status: {resp.status_code} | saved: {dst}')
            print(f'time: {t1} | delta: {delta} | eta: {eta}')


[1 of 10,290]
status: 200 | saved: /home/qcx201/Projects/CAtrade/raw/CA-1990-CA.csv
time: 2025-04-21 01:38:48.427346 | delta: 0:00:00.725847 | eta: 2:04:28.239783
[2 of 10,290]
status: 200 | saved: /home/qcx201/Projects/CAtrade/raw/CA-1990-CB.csv
time: 2025-04-21 01:38:49.127936 | delta: 0:00:01.426437 | eta: 2:02:17.586784
[3 of 10,290]
status: 200 | saved: /home/qcx201/Projects/CAtrade/raw/CA-1990-CC.csv
time: 2025-04-21 01:38:49.824697 | delta: 0:00:02.123198 | eta: 2:01:20.449371
[4 of 10,290]
status: 200 | saved: /home/qcx201/Projects/CAtrade/raw/CA-1990-CD.csv
time: 2025-04-21 01:38:50.522696 | delta: 0:00:02.821197 | eta: 2:00:54.705514
[5 of 10,290]
status: 200 | saved: /home/qcx201/Projects/CAtrade/raw/CA-1990-CE.csv
time: 2025-04-21 01:38:51.195850 | delta: 0:00:03.494351 | eta: 1:59:47.877950
[6 of 10,290]
status: 200 | saved: /home/qcx201/Projects/CAtrade/raw/CA-1990-CF.csv
time: 2025-04-21 01:38:51.858762 | delta: 0:00:04.157263 | eta: 1:58:45.547068
[7 of 10,290]
status: 

KeyboardInterrupt: 

In [ ]:
print('-- Complete --')